# CIFAR-10 Deep Learning with MLOps

**Problem**: Classify images into 10 CIFAR-10 classes.  
**Assumptions**: RGB 32×32 inputs; single-label classification; CPU training.  
**Limitations**: Small CNN baseline; few epochs; accuracy is modest by design.  
**Dataset**: CIFAR-10 (60k images: 50k train, 10k test). Standard normalization used.


## Pipeline Overview
- **Model**: `SmallCNN` (see `src/model.py`).
- **Preprocessing**: Resize(32,32) → ToTensor → Normalize(mean=(0.4914,0.4822,0.4465), std=(0.247,0.243,0.261)).
- **Training & Tracking**: `python -m src.train` logs metrics/artifacts to **MLflow** (`./mlflow_tracking`).
- **Evaluation**: `python -m src.evaluate` prints `TEST_ACC` and logs to MLflow.
- **Versioning**: Code via **Git/GitHub**; data/artifacts via **DVC** (`data.dvc`, `dvc.yaml`, `dvc.lock`).
- **CI**: GitHub Actions runs `pytest` on push.
- **Serving**: **FastAPI** (`src/predict_service.py`) — `/predict` for inference, `/health` for health.
- **Container**: `Dockerfile` → `docker build -t cifar-service .` then `docker run -p 8000:8000 cifar-service`.
- **Monitoring**: Each request logs channel mean/std and simple drift flags to `logs/service.log`.


In [ ]:
# Show test accuracy by calling the evaluator
import subprocess, sys
completed = subprocess.run([sys.executable, '-m', 'src.evaluate'], capture_output=True, text=True)
print(completed.stdout.strip() or completed.stderr.strip())


## Evidence (CI / Tracking / Serving / Monitoring)

- **MLflow experiments / runs**  
  ![](screens/mlflow_runs.png)

- **Swagger UI**  
  ![](screens/swagger.png)

- **/predict result**  
  ![](screens/predict.png)

- **Monitoring log**  
  ![](screens/monitoring_log.png)


## Observations
- The baseline CNN achieves the reported **TEST_ACC** above (printed by the code cell).
- MLflow records params/metrics and the saved `model.pt` artifact.
- CI confirms tests pass (`tests/test_model.py`).
- The API `/predict` works and logs basic **drift flags** if channel stats deviate (>15%).
- DVC tracks dataset and model artifact versions; `dvc repro` can rerun the pipeline.
